# Water Potability Model Exploration

In [11]:
import mlflow
import mlflow.spark
from utilities.spark_utils import get_spark_session, load_data
from pipeline import feature_pipeline

import sys, os
sys.path.append('/home/jovyan/work')

In [12]:
spark = get_spark_session()
spark.sparkContext.setLogLevel('ERROR')

## Load the Training Data

In [13]:
file_path = '../data/water_potability_train.csv'
df = load_data(spark, file_path)
df.show(5)

+---+----+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+----------+
|_c0|  ph|          Hardness|            Solids|       Chloramines|           Sulfate|      Conductivity|    Organic_carbon|  Trihalomethanes|         Turbidity|Potability|
+---+----+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+----------+
|  0|null|  98.3679148956603| 28415.57583214058|10.558949998467961|  296.843207792478|505.24026927891407|12.882614472289333|85.32995534051292| 4.119087300328971|         1|
|  1|null|103.46475866009455| 27420.16742458204| 8.417305032089528|              null|485.97450045781375|11.351132730708514| 67.8699636759021| 4.620793451653219|         0|
|  2|null|108.91662923953173|14476.335695268315| 5.398162017711099|  281.198274407849| 512.2323064106689|15.013793389990155| 86.6714587

## Model Experimentation with MLFlow

In [14]:
experiment = 'iris_features'
mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
mlflow.set_experiment(experiment)

INFO: 'iris_features' does not exist. Creating a new experiment


In [15]:
with mlflow.start_run() as run:
    pass